
De forma sencilla se puede ver que la regla compuesta de Simpson compuesta $S_c(f)$ se escribe como:

$$S_c(f) = \displaystyle \frac{h}{3n} \left [ f(x_0) + f(x_n) + 2 \sum_{i=1}^{\frac{n}{2}-1}f(x_{2i}) + 4 \sum_{i=1}^{\frac{n}{2}}f(x_{2i-1})\right ] $$
con $h=b-a$ y $n$ número de subintervalos (par).

Nota: Los nodos para el caso de Simpson se obtienen con la fórmula: $x_i = a +\frac{i}{2}\hat{h}, \forall i=0,\dots,2n, \hat{h}=\frac{h}{n}$.

Lo siguiente sirve de ayuda. Consideramos un caso de $n=6$ subintervalos. Estos índices se multiplican por 4 y son los índices que el subproceso 0 trabajará:

In [1]:
begin=1
end=6

In [2]:
for i in range(begin,end+1,2):
    print(i)

1
3
5


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 0 trabajará:

In [3]:
for i in range(begin+1,end+1,2):
    print(i)

2
4
6


In [4]:
begin=7
end=12

Los siguientes índices que se multiplican por 4 y son los índices que el subproceso 1 trabajará:

In [5]:
for i in range(begin,end+1,2):
    print(i)

7
9
11


Estos son los siguientes índices que se multiplican por 2 y son los índices que el subproceso 1 trabajará:

In [7]:
for i in range(begin+1,end+1,2):
    print(i)

8
10
12


In [29]:
import math
from multiprocessing import Pool 
from scipy.integrate import quad
import time
import multiprocessing as mp

**¿Qué restricciones se deben colocar a las variables $n, ns\text{_}p$ y $p$ para que el programa siguiente obtenga resultados correctos?**

En el caso de $p$, que es el número de subprocesos a lanzar, éste se recomienda que sea menor o igual al número de cores que el sistema operativo puede usar.
Para conocer el número de cores se utiliza la función cpu_count de multiprocessing.

In [31]:
import multiprocessing

In [32]:
multiprocessing.cpu_count()

4

A la variable $n$, número de subintervalos, se le debe colocar como restricción que sea par, pues es una restricción en la fórmula de la regla de Simpson.

En cuanto a $ns\_p$, el número de subintervalos que procesa cada subproceso, se recomienda repartirlos equitativamente entre cada subproceso.

---------------------------------------------------------------------------------------------------

por ejemplo si pruebas con $n=6, p=2, ns\text{_}p=6$ obtienes un error relativo de $10^{-7}$.

In [14]:
a=0
b=1
h=b-a
f=lambda x: math.exp(-x**2)

In [147]:
p=3 #número de subprocesos a lanzar

In [148]:
n=6 #número de subintervalos a lanzar
h_hat=h/n

In [149]:
nt=2*n #variable de apoyo para definir ns_p
ns_p=nt/p

In [150]:
nt

12

In [151]:
ns_p

4.0

In [152]:
ns_p=int(ns_p)

In [153]:
def Scf_parallel(mi_id):
    """
    Compute numerical approximation using Simpson rule in an interval
    Nodes are generated via formula: x_i = a+i/2*h_hat for i=0,1,...,2n and h_hat=(b-a)/n
    n must be an even number
    Args:
        mi_id (in): id of subprocess: 0,1,2,...
    """
    begin = (mi_id*ns_p)+1
    print("id")
    print(mi_id)
    print("begin")
    print(begin)
    end = begin-1+ns_p
    #print("end")
    #print(end)
    sum1=0
    #next for loop considers sum that is multiplied by 4 in the expression
    for i in range(begin,end+1,2):
        x = a+i/2*h_hat
        sum1+=f(x)
    sum1=4*sum1
    #print(sum1)
    sum2=0
    #next for loop considers sum that is multiplied by 2 in the expression
    for i in range(begin+1,end+1,2):
        x = a+i/2*h_hat
        sum2+=f(x)
    sum2=2*sum2
    #print(sum2)
    return sum1+sum2

if __name__=='__main__':
    start_time=time.time()
    with Pool(processes=p) as pool:
        results = pool.map(Scf_parallel,range(p))
        aprox=h/(6*n)*(f(a)+sum(results)-f(b)) 
    end_time=time.time()

id
id
id
1
0
2
begin
begin
1
5
begin
9


para dos subprocesos: uno inicia en 1 y termina en 6, el otro inicia en 7 y termina en 12

para tres subprocesos: uno inicia en 1 y termina en 4, otro inicia en 5 y termina en 8, y el otro incia en 9 y termina en 12

In [154]:
print([*range(1,4+1,2)])

[1, 3]


In [155]:
print([*range(1+1,4+1,2)])

[2, 4]


In [156]:
print([*range(5,8+1,2)])

[5, 7]


In [157]:
print([*range(5+1,8+1,2)])

[6, 8]


In [158]:
print([*range(9,12+1,2)])

[9, 11]


In [159]:
print([*range(9+1,12+1,2)])

[10, 12]


**En el bloque del with ¿por qué tenemos que dividir por $\frac{}{6n}$ en lugar de por $\frac{}{3n}$ y por qué tenemos que restar $f(b)$?**

Revisando a detalle la función *Scf_parallel*, se observa que se están repartiendo los nodos entre los subprocesos como nodos continuos de forma equitativa. Por ello, en la evaluación de nodos se incluye el último nodo de cada subintervalo (range termina en end+1) pues de lo contrario habría huecos en las evaluciones. Por ejemplo si $p=2$ y $n=6$, se reparten los nodos de 1 a 6 en un subproceso y de 7 a 12 en el otro subproceso, si no se incluyera end+1, no se estaría evaluando el nodo 6. 

De esta forma, el último nodo del intervalo completo, se está incluyendo en el for y por lo tanto ya se está evaluando el nodo $f(b)$, y no solo eso, además se está multiplicando por 2 el nodo $f(b)$ tal y como se hace con todos los nodos pares. Es decir, ya se incluye $2*f(b)$ en $sum2$. Por esto es necesario restar una vez $f(b)$ al final en $aprox$, para incluir solamente una vez el término $f(b)$ como lo indica la fórmula.

En el caso de $f(a)$, esto no ocurre pues el nodo cero no se incluye en ninguna evaluación de los for.

En cuanto a por qué se divide entre 6n en lugar de entre 3n

---------------------------------------------------------------------------------------------------

In [141]:
aprox

0.7468245263791944

In [142]:
obj, err = quad(f, a, b)

In [143]:
obj

0.7468241328124271

In [144]:
obj*2

1.4936482656248542

In [145]:
def err_relativo(aprox, obj):
    return math.fabs(aprox-obj)/math.fabs(obj) #obsérvese el uso de la librería math

In [146]:
err_relativo(aprox,obj)

5.269872115742658e-07

**¿Qué pasa si $n$ no es par? ¿qué tendríamos que modificar en la implementación si no es par?**